In [110]:
# uni_D83AE5_uni (Baseline)
#   {"Logloss": 0.024950720047991065, "ROCAUC": 0.9808700266292157}
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
from scipy.stats import gamma, kstest
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    log_loss, 
    roc_auc_score, 
    roc_curve, 
    auc
)
import statsmodels.api as sm
import warnings

warnings.filterwarnings('ignore')  # Use this line to suppress warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

training_data = pd.read_csv('training_data.csv')
training_data = training_data[training_data['BORROWER_ID'] != 'xNullx']
training_data = training_data.sample(frac=1, random_state=42).reset_index(drop=True)
data_submission_example = pd.read_csv('data_submission_example.csv')

lognormal_variables = [
    'CONTRACT_CREDIT_LOSS', 'CONTRACT_DEPT_SERVICE_TO_INCOME',
    'CONTRACT_INCOME', 'CONTRACT_INSTALMENT_AMOUNT', 'CONTRACT_INSTALMENT_AMOUNT_2',
    'CONTRACT_LOAN_AMOUNT', 'CONTRACT_MARKET_VALUE', 'CONTRACT_MORTGAGE_LENDING_VALUE', 
    'CONTRACT_LGD', 'CONTRACT_INCOME'
]

In [111]:
# # ADDITION 1
# import pandas as pd
# import numpy as np
# from sklearn.experimental import enable_iterative_imputer
# from sklearn.impute import IterativeImputer

# numeric_columns = training_data.select_dtypes(include=[np.number]).columns.tolist()
# excluded_keywords = ['TARGET', 'event', 'binary', 'DATE', 'DAYS', 'YEARS', 'MATURITY', 'DAY', 'BORROWER']

# X_columns = [col for col in numeric_columns if all(keyword.lower() not in col.lower() for keyword in excluded_keywords)]
# X_columns.remove('CONTRACT_TYPE_OF_INTEREST_REPAYMENT')
# X_columns.remove('CONTRACT_MORTGAGE_TYPE')
# df_copy = training_data.copy()
# print(df_copy.isna().sum().sum())
# min_values = []
# for i in X_columns:
#     if i in lognormal_variables:
#         min_values.append(1)
#     else:
#         min_values.append(-np.inf)
# imputer = IterativeImputer(max_iter=30, random_state=42, min_value=min_values,)
# imputed_values = imputer.fit_transform(df_copy[X_columns])

In [112]:
# # ADDITION 2
# imputed_df = pd.DataFrame(imputed_values, columns=X_columns)

# for col in X_columns:
#     if imputed_df[col].dtype == 'float64':
#         imputed_df[col] = imputed_df[col].astype('float32')
#     elif imputed_df[col].dtype == 'int64':
#         imputed_df[col] = imputed_df[col].astype('int32')

# training_data.update(imputed_df)

In [113]:
training_data.fillna(0, inplace=True)
for col in training_data.columns:
    try:
        training_data[col] = pd.to_numeric(training_data[col], errors='ignore')
    except:
        continue
training_data['TARGET_EVENT_BINARY'] = np.where(training_data['TARGET_EVENT'] == 'K', 1, 0)

In [114]:
training_data.isna().sum()

CONTRACT_ID                            0
BORROWER_ID                            0
CONTRACT_BANK_ID                       0
CONTRACT_CREDIT_INTERMEDIARY           0
CONTRACT_CREDIT_LOSS                   0
CONTRACT_CURRENCY                      0
CONTRACT_DATE_OF_LOAN_AGREEMENT        0
CONTRACT_DEPT_SERVICE_TO_INCOME        0
CONTRACT_FREQUENCY_TYPE                0
CONTRACT_INCOME                        0
CONTRACT_INSTALMENT_AMOUNT             0
CONTRACT_INSTALMENT_AMOUNT_2           0
CONTRACT_INTEREST_PERIOD               0
CONTRACT_INTEREST_RATE                 0
CONTRACT_LGD                           0
CONTRACT_LOAN_AMOUNT                   0
CONTRACT_LOAN_CONTRACT_TYPE            0
CONTRACT_LOAN_TO_VALUE_RATIO           0
CONTRACT_LOAN_TYPE                     0
CONTRACT_MARKET_VALUE                  0
CONTRACT_MATURITY_DATE                 0
CONTRACT_MORTGAGE_LENDING_VALUE        0
CONTRACT_MORTGAGE_TYPE                 0
CONTRACT_REFINANCED                    0
CONTRACT_RISK_WE

In [115]:
training_data['TARGET_EVENT_E'] = np.where(training_data['TARGET_EVENT'] == 'E', 1, 0)

In [116]:
training_data['TARGET_EVENT_BINARY'].value_counts()

TARGET_EVENT_BINARY
0    1590792
1      10871
Name: count, dtype: int64

In [117]:

date_variables = ['CONTRACT_DATE_OF_LOAN_AGREEMENT', 'CONTRACT_MATURITY_DATE']

In [118]:
training_data['TARGET_EVENT_DAY'].replace(0.0, np.nan, inplace=True)
training_data['TARGET_EVENT_DAY_JULIAN'] = pd.to_datetime(training_data['TARGET_EVENT_DAY'], origin='julian', unit='D', errors='coerce')
training_data['TARGET_EVENT_DAY_DATETIME'] = pd.to_datetime(training_data['TARGET_EVENT_DAY_JULIAN'],  errors='coerce')

training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN'] = pd.to_datetime(training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT'], origin='julian', unit='D')
training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] = pd.to_datetime(training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN'],)

training_data['CONTRACT_MATURITY_DATE_JULIAN'] = pd.to_datetime(training_data['CONTRACT_MATURITY_DATE'], origin='julian', unit='D')
training_data['CONTRACT_MATURITY_DATE_DATETIME'] = pd.to_datetime(training_data['CONTRACT_MATURITY_DATE_JULIAN'])

In [119]:
training_data['DAY_DIFF'] = (training_data['TARGET_EVENT_DAY_DATETIME'] - training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME']).dt.days
training_data['DAYS_TO_END'] = (pd.Timestamp("2020-01-01")- training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME']).dt.days
training_data['YEARS_TO_END'] = training_data['DAYS_TO_END'] / 365
training_data['DAYS_TO_2018'] = (pd.Timestamp("2018-01-01")- training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME']).dt.days
training_data['YEARS_TO_2018'] = training_data['DAYS_TO_2018'] / 365
training_data['TIME_TO_MATURITY_DAYS'] = (training_data['CONTRACT_MATURITY_DATE']-training_data['CONTRACT_DATE_OF_LOAN_AGREEMENT'])
training_data['2020_OR_MATURITY'] = np.minimum(training_data['TIME_TO_MATURITY_DAYS'], training_data['DAYS_TO_END'])
training_data['2020_OR_MATURITY_YEARS'] = training_data['2020_OR_MATURITY'] / 365

In [120]:
def create_binary_target_column(dataframe, column_name, event, day_diff_upper_limit):
    dataframe[column_name] = np.where(
        (dataframe['TARGET_EVENT'] == event) & 
        (dataframe['DAY_DIFF'] <= day_diff_upper_limit) & 
        (dataframe['DAY_DIFF'] >= 0), 
        1, 
        0
    )

timeframes = {
    'TARGET_EVENT_BINARY_2Y': 730,
    'TARGET_EVENT_BINARY_1Y': 365,
    'TARGET_EVENT_BINARY_6M': 365//2,
}

for column_name, days in timeframes.items():
    create_binary_target_column(training_data, column_name, 'K', days)


# target_event_binary_columns = []
# for i in range(6):
#     start_day = 273 + i * 30
#     column_header = 'TARGET_EVENT_BINARY_' + str(start_day) + 'D'
#     training_data[column_header] = np.where(
#         (training_data['TARGET_EVENT'] == 'K') & 
#         (training_data['DAY_DIFF'] <= start_day) & 
#         (training_data['DAY_DIFF'] >= 0), 
#         1, 
#         0
#     )
#     target_event_binary_columns.append(column_header)

training_data.drop('DAY_DIFF', axis=1, inplace=True)

In [121]:
print(training_data['TARGET_EVENT_BINARY'].value_counts())
print(training_data['TARGET_EVENT_BINARY_2Y'].value_counts())
print(training_data['TARGET_EVENT_BINARY_1Y'].value_counts())
print(training_data['TARGET_EVENT_BINARY_6M'].value_counts())

TARGET_EVENT_BINARY
0    1590792
1      10871
Name: count, dtype: int64
TARGET_EVENT_BINARY_2Y
0    1591751
1       9912
Name: count, dtype: int64
TARGET_EVENT_BINARY_1Y
0    1596927
1       4736
Name: count, dtype: int64
TARGET_EVENT_BINARY_6M
0    1601267
1        396
Name: count, dtype: int64


In [122]:
numeric_columns = training_data.select_dtypes(include=[np.number]).columns.tolist()
excluded_keywords = ['TARGET', 'event', 'binary', 'DATE', 'DAYS', 'YEARS', 'MATURITY', 'DAY']

X_columns = [col for col in numeric_columns if all(keyword.lower() not in col.lower() for keyword in excluded_keywords)]
y_column = 'TARGET_EVENT_BINARY_2Y' 
X_columns

['CONTRACT_CREDIT_INTERMEDIARY',
 'CONTRACT_CREDIT_LOSS',
 'CONTRACT_CURRENCY',
 'CONTRACT_DEPT_SERVICE_TO_INCOME',
 'CONTRACT_INCOME',
 'CONTRACT_INSTALMENT_AMOUNT',
 'CONTRACT_INSTALMENT_AMOUNT_2',
 'CONTRACT_INTEREST_PERIOD',
 'CONTRACT_INTEREST_RATE',
 'CONTRACT_LGD',
 'CONTRACT_LOAN_AMOUNT',
 'CONTRACT_LOAN_CONTRACT_TYPE',
 'CONTRACT_LOAN_TO_VALUE_RATIO',
 'CONTRACT_MARKET_VALUE',
 'CONTRACT_MORTGAGE_LENDING_VALUE',
 'CONTRACT_MORTGAGE_TYPE',
 'CONTRACT_REFINANCED',
 'CONTRACT_RISK_WEIGHTED_ASSETS',
 'CONTRACT_TYPE_OF_INTEREST_REPAYMENT',
 'BORROWER_BIRTH_YEAR',
 'BORROWER_CITIZENSHIP',
 'BORROWER_COUNTRY',
 'BORROWER_COUNTY',
 'BORROWER_TYPE_OF_SETTLEMENT']

In [123]:
import numpy as np
import numpy_financial as npf
import pandas as pd

def calculate_monthly_payment(row):
    annual_interest_rate = row['CONTRACT_INTEREST_RATE'] / 100  # Convert percentage to decimal
    monthly_interest_rate = annual_interest_rate / 12
    term_in_months = ((pd.to_datetime(row['CONTRACT_MATURITY_DATE_DATETIME'], format='%d/%m/%Y') - 
                       pd.to_datetime(row['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'], format='%d/%m/%Y'))).days/30
    principal = row['CONTRACT_LOAN_AMOUNT']
    monthly_payment = npf.pmt(monthly_interest_rate, term_in_months, -principal)
    return monthly_payment
# ADDITION 3:
# training_data['MONTHLY_PAYMENT'] = training_data.apply(calculate_monthly_payment, axis=1)
# training_data['MONTHLY_PAYMENT'] = training_data['MONTHLY_PAYMENT'].apply(lambda x: 0 if not np.isfinite(x) else x)
# X_columns.extend(['MONTHLY_PAYMENT'])
# lognormal_variables.extend(['MONTHLY_PAYMENT'])
# X_columns.append('TIME_TO_MATURITY_DAYS')

In [124]:
loan_type_dummies = pd.get_dummies(training_data['CONTRACT_LOAN_TYPE'], prefix='LOAN_TYPE', drop_first=True)
frequency_type_dummies = pd.get_dummies(training_data['CONTRACT_FREQUENCY_TYPE'], prefix='FREQ_TYPE', drop_first=True)
interest_type_dummies = pd.get_dummies(training_data['CONTRACT_TYPE_OF_INTEREST_REPAYMENT'], prefix='INTEREST_TYPE', drop_first=True)
mortgage_type_dummies =  pd.get_dummies(training_data['CONTRACT_MORTGAGE_TYPE'], prefix='MORTGAGE_TYPE', drop_first=True)

training_data = pd.concat([training_data, loan_type_dummies, frequency_type_dummies,interest_type_dummies,mortgage_type_dummies ], axis=1)

X_columns.extend(loan_type_dummies.columns)
X_columns.extend(frequency_type_dummies.columns)
X_columns.extend(interest_type_dummies.columns)
X_columns.extend(mortgage_type_dummies.columns)

In [125]:
training_data['BORROWER_LOAN_COUNT'] = training_data.groupby('BORROWER_ID')['BORROWER_ID'].transform('count')
training_data['LOAN_BORROWER_COUNT'] = training_data.groupby('CONTRACT_ID')['CONTRACT_ID'].transform('count')
training_data['TOTAL_LOAN_AMOUNT'] = training_data.groupby('BORROWER_ID')['CONTRACT_LOAN_AMOUNT'].transform('sum')
training_data['TOTAL_INSTALLMENT_AMOUNT_1'] = training_data.groupby('BORROWER_ID')['CONTRACT_INSTALMENT_AMOUNT'].transform('sum')
training_data['TOTAL_INSTALLMENT_AMOUNT_2'] = training_data.groupby('BORROWER_ID')['CONTRACT_INSTALMENT_AMOUNT_2'].transform('sum')
training_data['TOTAL_INSTALLMENT_AMOUNT'] = training_data['TOTAL_INSTALLMENT_AMOUNT_1'] + training_data['TOTAL_INSTALLMENT_AMOUNT_2']

X_columns.extend(['BORROWER_LOAN_COUNT', 'TOTAL_LOAN_AMOUNT','TOTAL_INSTALLMENT_AMOUNT','LOAN_BORROWER_COUNT'])
lognormal_variables.extend([ 'TOTAL_LOAN_AMOUNT','TOTAL_INSTALLMENT_AMOUNT'])

In [126]:
threshold = 0.95
correlation_matrix = training_data[X_columns].corr()
highly_correlated_set = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            highly_correlated_set.add(colname)

X_columns = [col for col in X_columns if col not in highly_correlated_set]
print('Variables removed:', highly_correlated_set)

Variables removed: {'BORROWER_COUNTRY', 'FREQ_TYPE_2f88e16c', 'CONTRACT_REFINANCED'}


In [127]:
def default_percentage_generator_2016(df, percentage, target):
    df_copy = df.copy()
    df_filtered = df_copy[df_copy['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] < '2016-01-01']
    df_mean = df_filtered[target].mean()
    print(f"Mean in year {df_mean}")
    df_defautled = df_filtered[target].sum()
    df_not_defaulted = len(df_filtered) - df_defautled

    required_val = (df_defautled - percentage * len(df_filtered))/(percentage - 1)
    
    df_filtered_after = df_copy[df_copy['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] > '2016-01-01']
    df_filtered_after = df_filtered_after[df_filtered_after[target]==1] 
    print(len(df_filtered_after))
    print(required_val)
    required_val = min(int(required_val) ,len(df_filtered_after))
    df_filtered_after = df_filtered_after.sample(n=int(required_val),random_state=42).reset_index(drop=True)

    df_filtered = pd.concat([df_filtered, df_filtered_after])

    return df_filtered

def default_percentage_generator_2016_maximal(df, percentage, target):
    df_copy = df.copy()

    defaulted = df_copy[(df_copy[target]==1) & (df_copy['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] < '2018-01-01')]
    total_needed = len(defaulted) / percentage * 100 - len(defaulted)


    # print(len(defaulted))
    # print(total_needed)

    df_filtered_2016 = df_copy[(df_copy['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] < '2016-01-01')& (df_copy[target]==0)]
    # print(len(df_filtered_2016))
    # print(total_needed)
    max_needed = min(total_needed,len(df_filtered_2016))
    df_filtered_2016 = df_filtered_2016.sample(n=int(max_needed),random_state=42).reset_index(drop=True)

    extra_needed = total_needed - len(df_filtered_2016)

    df_filtered_after = df_copy[df_copy['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] > '2016-01-01']
    df_filtered_after = df_filtered_after[df_filtered_after[target]==0]
    df_filtered_after = df_filtered_after.sample(n=int(extra_needed),random_state=42).reset_index(drop=True)


    df_filtered = pd.concat([df_filtered_2016, df_filtered_after,defaulted])

    return df_filtered

# default_percentage_generator_2016_maximal(training_data, 1.48, 'TARGET_EVENT_BINARY')['TARGET_EVENT_BINARY'].mean()


In [128]:
def calculate_probabilities(data, column, time_factor):
    lambdas = -np.log(1 - data[column]) / time_factor
    probs_2y = 1 - np.exp(-2 * lambdas)
    return probs_2y
def calculate_probabilities_vec(data, time_factor):
    lambdas = -np.log(1 - data) / time_factor
    probs_2y = 1 - np.exp(-2 * lambdas)
    return probs_2y


In [129]:
from sklearn.calibration import CalibratedClassifierCV
from imblearn.over_sampling import SMOTE


def train_and_predict_two_halves(df, variables, target, model=LogisticRegression(), scaler=StandardScaler(), augment_distribution=False,calibrate=False,augment_distribution_percentage = 1.48, unique_loans=False,should_smote =False,maximal_sample=False):
    if lognormal_variables is not None:
        df = df.copy()
        
        for var in lognormal_variables:
            if var == 'CONTRACT_CREDIT_LOSS':
                df[var] = np.log1p(np.abs(df[var]))*np.sign(df[var])
            else:
                df[var] = np.log1p(df[var])
    if augment_distribution and not maximal_sample:
        df_filtered = default_percentage_generator_2016(df, augment_distribution_percentage/100, target)
    else:
        df_filtered = df[df['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] < '2016-01-01']
    
    if maximal_sample:
        df_filtered = default_percentage_generator_2016_maximal(df, augment_distribution_percentage, target)
        print(df_filtered[target].mean())

    if unique_loans:
        df_filtered = df_filtered.drop_duplicates(subset=['CONTRACT_ID'])

    
    X_scaled = scaler.fit_transform(df[variables])
    X_filtered = scaler.transform(df_filtered[variables])

    y = df[target] 
    y_filtered = df_filtered[target] 
    if should_smote:
        smote = SMOTE(random_state=42)
        X_filtered, y_filtered = smote.fit_resample(X_filtered, y_filtered)

    if calibrate:
        model = CalibratedClassifierCV(base_estimator=model, method='isotonic', )

    model.fit(X_filtered, y_filtered)
    

    print("Logloss:")
    test_proba = model.predict_proba(X_filtered)
    print(log_loss(y_filtered,test_proba))
    proba = model.predict_proba(X_scaled)[:, 1]
    # Additional code for ROC AUC score
    roc_auc = roc_auc_score(y_filtered,test_proba[:,1])
    print("ROC AUC Score:", roc_auc)

    return proba


In [130]:
def significant_features(df, variables, target, model1=LogisticRegression(), scaler=StandardScaler()):
    if lognormal_variables is not None:
        df = df.copy()
        print(df[variables].isna().sum().sum())
        
        for var in lognormal_variables:
            if var == 'CONTRACT_CREDIT_LOSS':
                df[var] = np.log1p(np.abs(df[var]))*np.sign(df[var])
            else:
                df[var] = np.log1p(df[var])
    print(np.isinf(df[variables]).sum().sum())
    X_scaled = scaler.fit_transform(df[variables])
    print(np.isinf(X_scaled).sum().sum())
    print(pd.DataFrame(X_scaled, columns=variables).isna().sum().sum())
    X = sm.add_constant(pd.DataFrame(X_scaled, columns=variables))
    y = df[target]
    model = sm.Logit(y, X).fit()
    print(model.summary())
    p_values = model.pvalues
    non_significant_vars = p_values[p_values > 0.05].index.tolist()

    return model, non_significant_vars

In [131]:
def combined_probability(s):
    if len(s) == 2:
        p_a, p_b = s.values
        return p_a + p_b - p_a * p_b #- (-3.2357827075016176e-05)
    else:
        return 1 - np.prod(1 - s.values)

def create_submission_file(df_preds, target, example, filename='submission.csv', testing=False):
    # Filter the data to only include BORROWER_IDs that are in the submission example
    df_preds.loc[df_preds['TARGET_EVENT'] == 'E', target] = 0

    print("Logloss:")
    print(log_loss(df_preds['TARGET_EVENT_BINARY'], df_preds[target]))

    filtered_training_data = df_preds[df_preds['BORROWER_ID'].isin(example['BORROWER_ID'])]

    # Print warning if the row count is off
    if not testing and len(filtered_training_data) != 1564601:
        print('WARNING: The filtered data does not have the correct number of rows. Make sure you are not using the training data for submission.')
        # raise ValueError('WARNING: The submission file does not have the correct number of rows. Make sure you are not using the training data for submission.')
    # Group by BORROWER_ID and calculate the combined probability

    #######################x########################
    #CUTTING TAILS DID NOT SEEM TO WORK
    #######################x########################
    # filtered_training_data = cut_exponential_tails(filtered_training_data, target)
    grouped_data = filtered_training_data.groupby('BORROWER_ID')[target].apply(combined_probability).reset_index()

    # Create the submission DataFrame
    df_submission = pd.DataFrame()
    df_submission['BORROWER_ID'] = grouped_data['BORROWER_ID']
    df_submission['PRED'] = grouped_data[target]
    print('Before centering:')
    print(df_submission['PRED'].max(), df_submission['PRED'].min(), df_submission['PRED'].mean())

    print('Centering probabilities...')
    # Center the probabilities around 1.48%
    desired_mean = 0.0148  # 1.48% as a decimal
    # while (df_submission['PRED'].max() > 1 or df_submission['PRED'].min() < 0 or abs(df_submission['PRED'].mean() -0.0148) > 0.0005):
    #     # print(df_submission['PRED'].max(), df_submission['PRED'].min(), df_submission['PRED'].mean())
    #     df_submission['PRED'] = df_submission['PRED'].clip(lower=0, upper=1)
    #     # print(df_submission['PRED'].max(), df_submission['PRED'].min(), df_submission['PRED'].mean())

    #     current_mean = df_submission['PRED'].mean()
    #     adjustment_factor = desired_mean  - current_mean
    #     df_submission['PRED'] += adjustment_factor
    initial_guess = 2
    probas_unscaled = df_submission['PRED'].values
    new_proba = probas_unscaled.copy()
    while abs(new_proba.mean() - desired_mean) > 0.00001:
        
        new_proba = calculate_probabilities_vec(probas_unscaled, initial_guess)
        error = new_proba.mean() - desired_mean
        if error > 0:
            initial_guess += 0.001
        else:
            initial_guess -= 0.001
        print(error, initial_guess)
    df_submission['PRED'] = new_proba
        
        
    
    print(df_submission['PRED'].max(), df_submission['PRED'].min(), df_submission['PRED'].mean())
    # Save the submission file
    if  not testing and filename is not None:
        df_submission.to_csv(filename, index=False)
    print(f'Saved file: {filename}')
    # if abs(df_submission['PRED'].mean() -0.0148) > 0.0005:
    #    raise ValueError('WARNING: mean is bad')
        
    # Print warning if the row count is off
    if not testing and len(df_submission) != 1117674:
        print('WARNING: The submission file does not have the correct number of rows. Make sure you are not using the training data for submission.')
        # raise ValueError('WARNING: The submission file does not have the correct number of rows. Make sure you are not using the training data for submission.')
        
    return df_submission


In [132]:
def test_model_2y_1y(df, variables, target, model1=LogisticRegression(), model2=LogisticRegression()):
    df = df.copy()
    start_date = pd.Timestamp('2015-01-01')
    end_date = pd.Timestamp('2017-01-01')

    # Mask for rows with CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN between start_date and end_date
    mask_date_range = (df['CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN'] >= start_date) & (df['CONTRACT_DATE_OF_LOAN_AGREEMENT_JULIAN'] <= end_date)

    df = df[mask_date_range]

    probs = train_and_predict_two_halves(
        df, 
        variables, 
        target, 
        model1=model1,
        model2=model2,
    )
    df['2Y-1Y-PROBS'] = probs
    
    test_data = pd.read_csv('./data/1y-test.csv')


    df_submission = create_submission_file(df, '2Y-1Y-PROBS', test_data, filename=None, testing=True)

    merged_df = pd.merge(test_data, df_submission, on='BORROWER_ID')
    true_labels = merged_df['TARGET_EVENT_BINARY_1Y']
    predicted_probs = merged_df['PRED']
    logloss = log_loss(true_labels, predicted_probs)

    print(f'Log loss: {logloss}')
        
    return logloss

In [133]:
def scale_yearly_proba(data, proba, targets =[ 0.0051, 0.0098, 0.0185], logging=False):
    data = data.copy()
    probs = data[proba]
    starter_scales=[2.8, 1.2, 0.75]
    new_proba = np.zeros(len(data))
    mask_2016 = (data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] < '2016-01-01') & (data['TARGET_EVENT_BINARY'] != 1)
    mask_2017 = (data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] > '2016-01-01') & (data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] < '2017-01-01')& (data['TARGET_EVENT_BINARY'] != 1)
    mask_2018 = (data['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] > '2017-01-01') & (data['TARGET_EVENT_BINARY'] != 1)

    proba_2016 = probs
    proba_2017 = probs
    proba_2018 = probs
    if logging:
        print("Before scaling:")
        print(proba_2016[mask_2016].mean(), proba_2017[mask_2017].mean(), proba_2018[mask_2018].mean())
        print(probs.mean())
    calib_2016 = True
    calib_2017 = True
    calib_2018 = True

    while calib_2016 or calib_2017 or calib_2018:    
        proba_2016 = probs
        proba_2017 = probs
        proba_2018 = probs
        proba_2016 = calculate_probabilities_vec(proba_2016, starter_scales[0])
        proba_2017 = calculate_probabilities_vec(proba_2017, starter_scales[1])
        proba_2018 = calculate_probabilities_vec(proba_2018, starter_scales[2])
        
        new_proba[mask_2016] = proba_2016[mask_2016]
        new_proba[mask_2017] = proba_2017[mask_2017]
        new_proba[mask_2018] = proba_2018[mask_2018]
        if logging:
            print('Adter scaling:')
            print(proba_2016[mask_2016].mean(), proba_2017[mask_2017].mean(), proba_2018[mask_2018].mean())
            print(new_proba.mean())

        mean_2016 = proba_2016[mask_2016].mean()
        mean_2017 = proba_2017[mask_2017].mean()
        mean_2018 = proba_2018[mask_2018].mean()

        diff_2016 = mean_2016 - targets[0]
        diff_2017 = mean_2017 - targets[1]
        diff_2018 = mean_2018 - targets[2]
        if diff_2016 > 0.0001:
            starter_scales[0] += 0.01
        elif diff_2016 < -0.0001:
            starter_scales[0] -= 0.01
        else:
            calib_2016 = False
        if diff_2017 > 0.0001:
            starter_scales[1] += 0.01
        elif diff_2017 < -0.0001:
            starter_scales[1] -= 0.01
        else:
            calib_2017 = False
        if diff_2018 > 0.0001:
            starter_scales[2] += 0.01
        elif diff_2018 < -0.0001:
            starter_scales[2] -= 0.01
        else:
            calib_2018 = False
    return new_proba
    

In [134]:
# Warning: Maximum number of iterations has been exceeded.
#          Current function value: 0.014855
#          Iterations: 35
#                              Logit Regression Results                             
# ==================================================================================
# Dep. Variable:     TARGET_EVENT_BINARY_2Y   No. Observations:              1601663
# Model:                              Logit   Df Residuals:                  1601611
# Method:                               MLE   Df Model:                           51
# Date:                    Wed, 08 Nov 2023   Pseudo R-squ.:                  0.6053
# Time:                            17:03:09   Log-Likelihood:                -23792.
# converged:                          False   LL-Null:                       -60284.
# Covariance Type:                nonrobust   LLR p-value:                     0.000

#  No dummies and feature engineering:
# Optimization terminated successfully.
#          Current function value: 0.017321
#          Iterations 13
#                              Logit Regression Results                             
# ==================================================================================
# Dep. Variable:     TARGET_EVENT_BINARY_2Y   No. Observations:              1601663
# Model:                              Logit   Df Residuals:                  1601638
# Method:                               MLE   Df Model:                           24
# Date:                    Wed, 08 Nov 2023   Pseudo R-squ.:                  0.5398
# Time:                            20:38:24   Log-Likelihood:                -27743.
# converged:                           True   LL-Null:                       -60284.
# Covariance Type:                nonrobust   LLR p-value:                     0.000
# Two more variables added: LOAN_TYPE, FREQ_TYPE, MORGTAGE_TYPE, INTEREST_TYPE
# Warning: Maximum number of iterations has been exceeded.
#          Current function value: 0.014613
#          Iterations: 35
#                              Logit Regression Results                             
# ==================================================================================
# Dep. Variable:     TARGET_EVENT_BINARY_2Y   No. Observations:              1601663
# Model:                              Logit   Df Residuals:                  1601589
# Method:                               MLE   Df Model:                           73
# Date:                    Thu, 09 Nov 2023   Pseudo R-squ.:                  0.6118
# Time:                            18:28:24   Log-Likelihood:                -23405.
# converged:                          False   LL-Null:                       -60284.
# Covariance Type:                nonrobust   LLR p-value:                     0.000
# Adding monthly to LOAN_TYPE, FREQ_TYPE, MORGTAGE_TYPE, INTEREST_TYPE
# Warning: Maximum number of iterations has been exceeded.
#          Current function value: 0.014232
#          Iterations: 35
#                              Logit Regression Results                             
# ==================================================================================
# Dep. Variable:     TARGET_EVENT_BINARY_2Y   No. Observations:              1601663
# Model:                              Logit   Df Residuals:                  1601584
# Method:                               MLE   Df Model:                           78
# Date:                    Thu, 09 Nov 2023   Pseudo R-squ.:                  0.6219
# Time:                            18:35:37   Log-Likelihood:                -22795.
# converged:                          False   LL-Null:                       -60284.
# Covariance Type:                nonrobust   LLR p-value:                     0.000
#  ADDING TIME TO MATURITY in days
# Warning: Maximum number of iterations has been exceeded.
#          Current function value: 0.014143
#          Iterations: 35
#                              Logit Regression Results                             
# ==================================================================================
# Dep. Variable:     TARGET_EVENT_BINARY_2Y   No. Observations:              1601663
# Model:                              Logit   Df Residuals:                  1601586
# Method:                               MLE   Df Model:                           76
# Date:                    Mon, 13 Nov 2023   Pseudo R-squ.:                  0.6242
# Time:                            17:00:42   Log-Likelihood:                -22653.
# converged:                          False   LL-Null:                       -60284.
# Covariance Type:                nonrobust   LLR p-value:                     0.000



In [135]:
X_copy = X_columns.copy()
X_copy.remove('CONTRACT_CREDIT_INTERMEDIARY')
X_copy.remove('BORROWER_COUNTY')
X_copy.remove('CONTRACT_TYPE_OF_INTEREST_REPAYMENT')
X_copy.remove('CONTRACT_MORTGAGE_TYPE')
X_copy.remove('BORROWER_TYPE_OF_SETTLEMENT')
X_copy.remove('CONTRACT_LOAN_AMOUNT')
X_copy.remove('CONTRACT_INSTALMENT_AMOUNT')


In [136]:
training_data[X_columns].isna().sum().sum()

0

In [137]:
model, non_significant_vars = significant_features(training_data, X_columns, y_column,LogisticRegression())

0
0
0
0
         Current function value: 0.014609
         Iterations: 35
                             Logit Regression Results                             
Dep. Variable:     TARGET_EVENT_BINARY_2Y   No. Observations:              1601663
Model:                              Logit   Df Residuals:                  1601588
Method:                               MLE   Df Model:                           74
Date:                    Wed, 22 Nov 2023   Pseudo R-squ.:                  0.6119
Time:                            11:04:31   Log-Likelihood:                -23398.
converged:                          False   LL-Null:                       -60284.
Covariance Type:                nonrobust   LLR p-value:                     0.000
                                          coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------
const                                 -10.3999   2.84e

In [138]:
non_significant_vars

['const',
 'CONTRACT_INSTALMENT_AMOUNT',
 'CONTRACT_LOAN_AMOUNT',
 'CONTRACT_MORTGAGE_TYPE',
 'CONTRACT_TYPE_OF_INTEREST_REPAYMENT',
 'BORROWER_COUNTY',
 'BORROWER_TYPE_OF_SETTLEMENT',
 'LOAN_TYPE_1f951336',
 'LOAN_TYPE_2f88e16c',
 'LOAN_TYPE_47693941',
 'LOAN_TYPE_5a06241e',
 'LOAN_TYPE_694cbaee',
 'LOAN_TYPE_69f70539',
 'LOAN_TYPE_7e2065f4',
 'LOAN_TYPE_83910425',
 'LOAN_TYPE_8fe006f1',
 'LOAN_TYPE_955ae3ef',
 'LOAN_TYPE_95c4f8fb',
 'LOAN_TYPE_b503a0de',
 'LOAN_TYPE_cde77491',
 'LOAN_TYPE_cf07c2dd',
 'LOAN_TYPE_d3aaffde',
 'LOAN_TYPE_eab72d7a',
 'LOAN_TYPE_f792971b',
 'FREQ_TYPE_89efd382',
 'FREQ_TYPE_bd092d5a',
 'INTEREST_TYPE_100001.0',
 'INTEREST_TYPE_100002.0',
 'INTEREST_TYPE_100003.0',
 'INTEREST_TYPE_100004.0',
 'INTEREST_TYPE_110001.0',
 'INTEREST_TYPE_140001.0',
 'INTEREST_TYPE_140002.0',
 'INTEREST_TYPE_140003.0',
 'MORTGAGE_TYPE_1.0',
 'MORTGAGE_TYPE_3.0',
 'MORTGAGE_TYPE_4.0',
 'MORTGAGE_TYPE_5.0',
 'MORTGAGE_TYPE_6.0',
 'MORTGAGE_TYPE_7.0',
 'MORTGAGE_TYPE_8.0',
 'MORTGA

In [139]:
X_columns.remove('CONTRACT_CURRENCY')
X_columns.remove('BORROWER_COUNTY')
X_columns.remove('CONTRACT_MORTGAGE_TYPE')
X_columns.remove('BORROWER_TYPE_OF_SETTLEMENT')
# Assuming you have a pandas DataFrame named training_data
# unique_training_data = training_data.drop_duplicates(subset='CONTRACT_ID', keep='first')


In [140]:
# 0.006785876080349979
# Before centering:
# 0.9978858840423693 8.548717289613705e-15 0.006063585227926563
# Centering probabilities...
# 1.0 0.008736424229876956 0.01479999999999999
# One extra dummy
# Logloss: 0.02502985804399272
# Zero extra dummy
# Logloss: 0.025941091599465484
# All dummies
# Logloss: 0.024974156733831323
# Adding calib (isotonic)
# Logloss: 0.02385418586559387
# Logloss: imputed
# 0.02372888852503455
# Logloss: imputed and removed a few unnecessary variables
# 0.023720691340422
# Logloss: imouted and added monthly payment
# 0.02294559159914238
predicted_probs = 'LOGISTIC_REG'
probs = train_and_predict_two_halves(
    training_data, 
    X_columns, 
    'TARGET_EVENT_BINARY',
    model=LogisticRegression(max_iter=400, random_state=42,solver='lbfgs'),
    augment_distribution=True,
    augment_distribution_percentage=1.48,
    calibrate=True,
    unique_loans=False,
    should_smote=False,
    maximal_sample=False,
)
training_data[predicted_probs] = probs
print(probs.mean())
training_data.loc[training_data['TARGET_EVENT'] == 'E', 'LOGISTIC_REG'] = 0

Mean in year 0.011345816067598232
5206
1750.5931790499392
Logloss:
0.023868488801544085
ROC AUC Score: 0.9884455664174394
0.01436353973362327


In [141]:
new_proba = scale_yearly_proba(training_data, 'LOGISTIC_REG')
print(new_proba.mean())
training_data['YEAR_SCALED_PROBA'] = new_proba

0.011471762617978111


In [142]:
submission = create_submission_file(training_data, 'YEAR_SCALED_PROBA', data_submission_example, filename='./predictions/best-model-so-far.csv')

Logloss:
0.26207997638270275
Before centering:
0.9999999623206586 6.3741058324851e-06 0.015031788522875091
Centering probabilities...
0.00023178852287509058 2.001
0.00022692203316808726 2.002
0.00022205890538514947 2.0029999999999997
0.00021719913576989137 2.0039999999999996
0.00021234272057200215 2.0049999999999994
0.0002074896560471315 2.0059999999999993
0.0002026399384570752 2.0069999999999992
0.0001977935640694456 2.007999999999999
0.00019295052915800116 2.008999999999999
0.00018811083000247995 2.009999999999999
0.00018327446288839318 2.010999999999999
0.0001784414241074294 2.0119999999999987
0.0001736117099571128 2.0129999999999986
0.00016878531674086386 2.0139999999999985
0.00016396224076810004 2.0149999999999983
0.0001591424783540761 2.0159999999999982
0.00015432602581992756 2.016999999999998
0.00014951287949272266 2.017999999999998
0.0001447030357053427 2.018999999999998
0.00013989649079654103 2.019999999999998
0.00013509324111088926 2.0209999999999977
0.00013029328299882757 2.

In [143]:
# Logloss:
# 0.010851681705608482
# ROC AUC Score: 0.9988352689711549
predicted_probs = 'XGB'
probs = train_and_predict_two_halves(
    training_data, 
    X_columns, 
    'TARGET_EVENT_BINARY',
    model=xgb.XGBClassifier(max_depth=2, n_estimators=70, random_state=42, use_label_encoder=False, eval_metric='logloss'),
    augment_distribution=True,
    augment_distribution_percentage=1.48,
    calibrate=True,
)
training_data[predicted_probs] = probs
print(probs.mean())
# this helps
training_data.loc[training_data['TARGET_EVENT'] == 'E', 'XGB'] = 0

Mean in year 0.011345816067598232
5206
1750.5931790499392
Logloss:
0.01727993881580623
ROC AUC Score: 0.9963451071460232
0.012088098393529596


In [144]:
training_data['ENSEMBLE'] = 0.4*training_data['XGB'] + 0.6*training_data['LOGISTIC_REG']
new_proba = scale_yearly_proba(training_data, 'ENSEMBLE')
training_data['XGB_YEAR_SCALED_PROBA'] = new_proba
print(new_proba.mean())

0.011420687278482304


In [145]:
submission = create_submission_file(training_data, 'XGB_YEAR_SCALED_PROBA', data_submission_example, filename='./predictions/best-model-so-far.csv')

Logloss:
0.263031275248133
Before centering:
0.9999999995735819 4.142014932706495e-06 0.014833470872319666
Centering probabilities...
3.347087231966514e-05 2.001
2.886188259912728e-05 2.002
2.4256032155318616e-05 2.0029999999999997
1.9653317456656133e-05 2.0039999999999996
1.5053734977378552e-05 2.0049999999999994
1.045728119745265e-05 2.0059999999999993
5.8639526025437705e-06 2.0069999999999992
0.9999999995451564 4.129626080096038e-06 0.014805863952602544
Saved file: ./predictions/best-model-so-far.csv


In [146]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

# Logloss:
# 0.02484109588442129 RF 200-3
# Logloss:
# 0.023534501248296504 RF 200-4-10
# Logloss:
# 0.02304888836026815 400-4- log_loss 25 
# Logloss:
# 0.0216212401349101 400-5- log_loss 25
# Logloss:
# 0.020320409332841818 400-6- log_loss 25
# Logloss:
# 0.018208761775049223 400-8- log_loss 30
# Logloss:
# 0.019426191288290054 200-8- log_loss 100
# Logloss:
# 0.01860565128621486 400 10 Logloss 100
# Logloss:
# 0.018445624629156568 400 11 Logloss 100

predicted_probs = 'NN'
probs = train_and_predict_two_halves(
    training_data, 
    X_columns, 
    'TARGET_EVENT_BINARY',
    model= RandomForestClassifier(n_estimators=600, max_depth=11, random_state=42, min_samples_leaf=100, criterion="log_loss", n_jobs=-1   ),
    augment_distribution=True,
    augment_distribution_percentage=1.48,
    calibrate=True,
)
training_data[predicted_probs] = probs
print(probs.mean())
# this helps
training_data.loc[training_data['TARGET_EVENT'] == 'E', 'NN'] = 0

Mean in year 0.011345816067598232
5206
1750.5931790499392
Logloss:
0.018344382184530682
ROC AUC Score: 0.9961497377868637
0.01241943327260914


In [147]:
training_data['ENSEMBLE_NN'] = 0.5*training_data['ENSEMBLE']+ 0.5*training_data['NN']

In [148]:
new_proba = scale_yearly_proba(training_data, 'ENSEMBLE_NN')
print(new_proba.mean())

0.011410706533720307


In [149]:
training_data['NN_LOG_XGB'] = new_proba

In [150]:
# Logloss:
# 0.2627792536515289: 
submission = create_submission_file(training_data, 'NN_LOG_XGB', data_submission_example, filename='./predictions/nn-xgb-log.csv')

Logloss:
0.26391921060636875
Before centering:
0.9999999999998295 2.2064184639347673e-06 0.01463195102982293
Centering probabilities...
-0.00016804897017707113 1.999
-0.00016364759303028747 1.9980000000000002
-0.0001592432506811025 1.9970000000000003
-0.0001548359397882306 1.9960000000000004
-0.00015042565700490625 1.9950000000000006
-0.00014601239897886475 1.9940000000000007
-0.00014159616235236497 1.9930000000000008
-0.00013717694376209552 1.9920000000000009
-0.0001327547398392824 1.991000000000001
-0.0001283295472095953 1.990000000000001
-0.000123901362493092 1.9890000000000012
-0.00011947018230437124 1.9880000000000013
-0.00011503600325238339 1.9870000000000014
-0.00011059882194050345 1.9860000000000015
-0.00010615863496651716 1.9850000000000017
-0.00010171543892258973 1.9840000000000018
-9.726923039528844e-05 1.9830000000000019
-9.282000596550107e-05 1.982000000000002
-8.836776220849316e-05 1.981000000000002
-8.391249569386638e-05 1.9800000000000022
-7.945420298554988e-05 1.979000

In [70]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

# Logloss:
# 0.02484109588442129 RF 200-3
predicted_probs = 'LGBM'
probs = train_and_predict_two_halves(
    training_data, 
    X_columns, 
    'TARGET_EVENT_BINARY',
    model= LGBMClassifier(n_estimators=60, max_depth=4,boosting_type="dart", random_state=42),
    augment_distribution=True,
    augment_distribution_percentage=1.48,
    calibrate=True,
)
training_data[predicted_probs] = probs
print(probs.mean())
# this helps
training_data.loc[training_data['TARGET_EVENT'] == 'E', 'LGBM'] = 0

training_data['ENSEMBLE_LGBM'] = 0.9*training_data['ENSEMBLE_NN']+ 0.1*training_data['LGBM']

Mean in year 0.011345816067598232
5206
1750.5931790499392
[LightGBM] [Info] Number of positive: 5932, number of negative: 394910
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055377 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4502
[LightGBM] [Info] Number of data points in the train set: 400842, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.014799 -> initscore=-4.198296
[LightGBM] [Info] Start training from score -4.198296
[LightGBM] [Info] Number of positive: 5932, number of negative: 394910
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.055007 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4506
[LightGBM] [Info] Number of data points in the train

In [71]:
new_proba = scale_yearly_proba(training_data, 'ENSEMBLE_LGBM')
print(new_proba.mean())
training_data['NN_LOG_XGB_LGBM'] = new_proba

0.011416816987101135


In [72]:
# Logloss:
# 0.2627792536515289: 
submission = create_submission_file(training_data, 'NN_LOG_XGB_LGBM', data_submission_example, filename='./predictions/nn-xgb-log-lgbm.csv')

Logloss:
0.2658451083533875
Before centering:
0.9999999999999993 7.133126343594753e-06 0.01454769895618425
Centering probabilities...
-0.0002523010438157513 1.999
-0.00024821121286087294 1.9980000000000002
-0.0002441187098878063 1.9970000000000003
-0.00024002353190681787 1.9960000000000004
-0.00023592567592323373 1.9950000000000006
-0.00023182513893748452 1.9940000000000007
-0.00022772191794510552 1.9930000000000008
-0.00022361600993668108 1.9920000000000009
-0.0002195074118978603 1.991000000000001
-0.00021539612080934308 1.990000000000001
-0.00021128213364687495 1.9890000000000012
-0.0002071654473812315 1.9880000000000013
-0.00020304605897822348 1.9870000000000014
-0.00019892396539863445 1.9860000000000015
-0.0001947991635982433 1.9850000000000017
-0.00019067165052788666 1.9840000000000018
-0.00018654142313326467 1.9830000000000019
-0.00018240847835514044 1.982000000000002
-0.00017827281312917005 1.981000000000002
-0.0001741344243859893 1.9800000000000022
-0.00016999330905114428 1.979

In [73]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
# XGB is the lowest score 
# Logloss:
# 0.01727067871139171
base_models = [
    ('rf', RandomForestClassifier(n_estimators=400, max_depth=11, random_state=42, min_samples_leaf=100, criterion="log_loss", n_jobs=-1   )),
    ('lgb', LGBMClassifier(n_estimators=60, max_depth=4,boosting_type="dart", random_state=42)),
    ('xgb', xgb.XGBClassifier(max_depth=2, n_estimators=70, random_state=42, use_label_encoder=False, eval_metric='logloss'))
]
meta_model = LogisticRegression()
stacked_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

predicted_probs = 'STACKED'
probs = train_and_predict_two_halves(
    training_data, 
    X_columns, 
    'TARGET_EVENT_BINARY',
    model= stacked_model,
    augment_distribution=True,
    augment_distribution_percentage=1.48,
    calibrate=True,
)
training_data[predicted_probs] = probs
print(probs.mean())
# this helps
training_data.loc[training_data['TARGET_EVENT'] == 'E', 'STACKED'] = 0

training_data['ENSEMBLE_STACKED'] = 0.9*training_data['ENSEMBLE_NN']+ 0.1*training_data['STACKED']

Mean in year 0.011345816067598232
5206
1750.5931790499392
[LightGBM] [Info] Number of positive: 5932, number of negative: 394910
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.049611 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4502
[LightGBM] [Info] Number of data points in the train set: 400842, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.014799 -> initscore=-4.198296
[LightGBM] [Info] Start training from score -4.198296
[LightGBM] [Info] Number of positive: 4745, number of negative: 315928
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.032953 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4505
[LightGBM] [Info] Number of data points in the train

In [ ]:
stacked_model.final_estimator_.coef_

In [51]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis



predicted_probs = 'QDA'
probs = train_and_predict_two_halves(
    training_data, 
    X_columns, 
    'TARGET_EVENT_BINARY',
    model= QuadraticDiscriminantAnalysis(),
    augment_distribution=True,
    augment_distribution_percentage=1.48,
    calibrate=True,
)
training_data[predicted_probs] = probs
print(probs.mean())
# this helps
training_data.loc[training_data['TARGET_EVENT'] == 'E', 'QDA'] = 0

training_data['ENSEMBLE_QDA'] = 0.9*training_data['ENSEMBLE_NN']+ 0.1*training_data['QDA']

Mean in year 0.011345816067598232
5206
1750.5931790499392
Logloss:
0.03345960035278355
ROC AUC Score: 0.9690328434892286
0.019327983772770608


In [74]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

# Logloss:
# 0.025080414519332874 1% & 10 estimators
predicted_probs = 'SVC'
probs = train_and_predict_two_halves(
    training_data, 
    X_columns, 
    'TARGET_EVENT_BINARY',
    model= BaggingClassifier(SVC(kernel='rbf', probability=True), max_samples=2000, n_estimators=100, n_jobs=-1, random_state=42),
    augment_distribution=True,
    augment_distribution_percentage=1.48,
    calibrate=True,
)
training_data[predicted_probs] = probs
print(probs.mean())
training_data.loc[training_data['TARGET_EVENT'] == 'E', 'SVC'] = 0
training_data['ENSEMBLE_SVC'] = 0.9*training_data['ENSEMBLE_LGBM']+ 0.1*training_data['SVC']

Mean in year 0.011345816067598232
5206
1750.5931790499392
Logloss:
0.029537779758320133


MemoryError: Unable to allocate 880. MiB for an array with shape (1601663, 72) and data type float64

In [ ]:
new_proba = scale_yearly_proba(training_data, 'ENSEMBLE_SVC')
print(new_proba.mean())
training_data['NN_LOG_XGB_LGBM_SVC'] = new_proba

In [ ]:
# Logloss:
# 0.2627792536515289: 
submission = create_submission_file(training_data, 'NN_LOG_XGB_LGBM_SVC', data_submission_example, filename='./predictions/nn-xgb-log-lgbm-svc.csv')

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import numpy as np

# Other necessary imports, like pandas, if you're using a DataFrame

def train_and_evaluate(df, variables, target, model=LogisticRegression(), scaler=StandardScaler(), cv_folds=5, augment_distribution=False):
    # Your lognormal_variables transformation code remains the same
    if lognormal_variables is not None:
        df = df.copy()
        for var in lognormal_variables:
            if var == 'CONTRACT_CREDIT_LOSS':
                df[var] = np.log1p(np.abs(df[var]))*np.sign(df[var])
            else:
                df[var] = np.log1p(df[var])

    # Split the data before the given date for training 
    if augment_distribution:
        df_filtered = default_percentage_generator_2016(df, 1.48/100, target)
    else:
        df_filtered = df[df['CONTRACT_DATE_OF_LOAN_AGREEMENT_DATETIME'] < '2016-01-01']
    X_filtered = df_filtered[variables]
    y_filtered = df_filtered[target]
    
    # Scale the filtered features
    X_filtered_scaled = scaler.fit_transform(X_filtered)
    
    # Perform cross-validation and fit the model
    cv_scores = cross_val_score(model, X_filtered_scaled, y_filtered, cv=cv_folds, scoring='neg_log_loss')
    
    print(f"CV Log Loss Scores: {-cv_scores}")
    print(f"Mean CV Log Loss: {-np.mean(cv_scores)}")
    
    # Now fit the model to the entire dataset
    model.fit(X_filtered_scaled, y_filtered)
    
    # Scale the entire dataset to predict probabilities for all records
    X_scaled = scaler.transform(df[variables])
    
    # Predict probabilities using the trained model
    proba = model.predict_proba(X_scaled)[:, 1]

    print("Logloss:")
    print(log_loss(y_filtered, model.predict_proba(X_filtered_scaled)))
    # Calculate log loss on the full dataset
    full_log_loss = log_loss(df[target], proba)

    print(f"Full Dataset Log Loss: {full_log_loss}")

    return proba, cv_scores, full_log_loss

# You might call the function like this
# probabilities, cv_scores, full_log_loss = train_and_evaluate(df, variables, target)



In [ ]:
# Logloss xgb :
# 0.016317211616370328
# Logloss logistic regression:
# 0.02385418586559387
predicted_probs = 'XGBOOST'
probs, _, _ = train_and_evaluate(
    training_data, 
    X_columns, 
    'TARGET_EVENT_BINARY',
    model=xgb.XGBClassifier(max_depth=3, reg_lambda=1, n_estimators=60, random_state=42, use_label_encoder=False, eval_metric='logloss'),
    augment_distribution=True
)
training_data[predicted_probs] = probs
print(probs.mean())


In [ ]:
probs.std()

In [ ]:
magic_number = 1.6
experiment_probs = calculate_probabilities(training_data, 'XGBOOST', magic_number)
print(experiment_probs.mean(), experiment_probs.max(), experiment_probs.min(), experiment_probs.std())


In [ ]:
training_data['EXPERIMENT_PROBS_XGB'] = experiment_probs
submission_exp = create_submission_file(training_data, 'EXPERIMENT_PROBS_XGB', data_submission_example, filename='./predictions/xgb-prediction-with-time-param-0-585.csv')

In [ ]:
# print(experiment_probs.mean(), experiment_probs.max(), experiment_probs.min(), experiment_probs.std())
sns.kdeplot(submission, label='Submission OG', lw=3, alpha=1)
sns.kdeplot(training_data['LOGISTIC_REG'], label='PredictionOG', lw=3, alpha=1)
sns.kdeplot(experiment_probs, label='Fine tuned solution', lw=3, alpha=1)
sns.kdeplot(submission_exp, label='Fine tuned solution per borrower', lw=3, alpha=1)

plt.yscale('log')
plt.legend()
plt.title('Overlayed Histogram of Submission and Predicted Probabilities')
plt.show()

In [ ]:
condition = (submission_exp['PRED'] > 0.9999)# | (submission_exp['PRED'] < 0.01)
ensemble_submission = submission.copy()
ensemble_submission.loc[condition,'PRED'] = submission_exp.loc[condition,'PRED']

In [ ]:
condition.sum()

In [ ]:
ensemble_submission['PRED'].mean()  

In [ ]:
calculate_probabilities_vec(ensemble_submission['PRED'].values, 2.029).mean()

In [ ]:
ensemble_submission['PRED'] = calculate_probabilities_vec(ensemble_submission['PRED'].values, 2.029)

In [ ]:
ensemble_submission.to_csv('./predictions/ensemble-submission.csv', index=False)

In [ ]:
ensemble_submission.head(5)

In [ ]:
# print(experiment_probs.mean(), experiment_probs.max(), experiment_probs.min(), experiment_probs.std())
sns.kdeplot(submission, label='Submission OG', lw=3, alpha=1, color='r')
# sns.kdeplot(training_data['LOGISTIC_REG'], label='PredictionOG', lw=3, alpha=1)
sns.kdeplot(ensemble_submission, label='Fine tuned solution', lw=3, alpha=1, color='g')
# sns.kdeplot(submission_exp, label='Fine tuned solution per borrower', lw=3, alpha=1)

plt.yscale('log')
plt.legend()
plt.title('Overlayed Histogram of Submission and Predicted Probabilities')
plt.show()